In this notebook, I build a model for music generation using techniques to process music files. The dataset contains MIDI files providing information about the music, including notes, chords, and instruments used
The dataset is available at the following link:

https://www.kaggle.com/datasets/ahemateja19bec1025/musicgenerationdataset/data

In [3]:
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from music21 import converter, instrument, note, chord
import glob

DATA_PATH = "/kaggle/input/musicgenerationdataset"

for root, dirs, files in os.walk(DATA_PATH):
    print(root, len(files))
    break


/kaggle/input/musicgenerationdataset 32


In [4]:
!pip install music21 tensorflow


# Load and parse MIDI files  to instruments and pitch (note ) and chord 

In [6]:



midi_files = glob.glob(DATA_PATH + "/**/*.mid", recursive=True)

notes = []

for file in midi_files:
    midi = converter.parse(file)
    parts = instrument.partitionByInstrument(midi)

    if parts:
        elements = parts.parts[0].recurse()
    else:
        elements = midi.flat.notes

    for el in elements:
        if isinstance(el, note.Note):
            notes.append(str(el.pitch))
        elif isinstance(el, chord.Chord):
            notes.append('.'.join(str(n) for n in el.normalOrder))


# Encoding notes

In [8]:
import numpy as np

unique_notes = sorted(set(notes))
note_to_int = {n:i for i,n in enumerate(unique_notes)}

sequence_length = 100
X, y = [], []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i+sequence_length]
    seq_out = notes[i+sequence_length]
    X.append([note_to_int[n] for n in seq_in])
    y.append(note_to_int[seq_out])

X = np.array(X)
y = np.array(y)

X = X / float(len(unique_notes))


#  Build the model using  LSTM 

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
    LSTM(512, input_shape=(X.shape[1], 1), return_sequences=True),
    Dropout(0.3),
    LSTM(512),
    Dense(256, activation='relu'),
    Dense(len(unique_notes), activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam'
)

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 100, 512)       │     1,052,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 512)            │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 153)            │        39,321 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,322,521 (12.67 MB)

 Trainable params: 3,322,521 (12.67 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
X = np.reshape(X, (X.shape[0], X.shape[1], 1))


# train the model

In [12]:
model.fit(
    X, y,
    epochs=70,
    batch_size=64
)


Epoch 1/70


I0000 00:00:1767801479.358714     134 cuda_dnn.cc:529] Loaded cuDNN version 91002


77/77 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - loss: 4.5290
Epoch 2/70
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 4.3194
Epoch 3/70
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 4.1231
Epoch 4/70
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 4.0151
Epoch 5/70
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 4.1447
Epoch 6/70
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 3.9447
Epoch 7/70
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 3.9211
Epoch 8/70
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 3.9112
Epoch 9/70
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 3.8590
Epoch 10/70
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 3.8712
Epoch 11/70
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 3.7830
Epoch 12/70
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 3.7491
Epoch 13/70
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 3.7111
Epoch 14/70
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 3.7091
Epoch 15/70
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 3.5781
Epoch 16/70
77/

# generate new music

In [14]:
import random

start = random.randint(0, len(X)-1)
pattern = X[start]

generated = []

for i in range(500):
    prediction = model.predict(pattern.reshape(1, pattern.shape[0], 1), verbose=0)
    index = np.argmax(prediction)
    generated.append(unique_notes[index])

    pattern = np.append(pattern, [[index / float(len(unique_notes))]], axis=0)
    pattern = pattern[1:]


# trensforms music generated to mid file

In [16]:

output = []
offset = 0

for n in generated:
    if '.' in n:   
        notes_in_chord = n.split('.')
        chord_notes = [note.Note(int(i) + 60) for i in notes_in_chord]
        new_chord = chord.Chord(chord_notes)
        new_chord.offset = offset
        output.append(new_chord)

    else:  
        if n.isdigit():           
            new_note = note.Note(int(n) + 60)
        else:                     
            new_note = note.Note(n)

        new_note.offset = offset
        output.append(new_note)

    offset += 0.5

midi_stream = stream.Stream(output)
midi_stream.write('midi', fp='generated_music.mid')


'generated_music.mid'

# backup the model

In [18]:

model.save('/kaggle/working/music_model.keras')

